In [67]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import joblib

## Load Dataset

In [41]:
df_data = pd.read_csv("../data/heart.xls")

In [42]:
df_data.shape

(303, 14)

In [43]:
df_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [44]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [45]:
df_data["target"].value_counts()

1    165
0    138
Name: target, dtype: int64

In [46]:
X, y = df_data.drop(columns=["target"]), df_data[["target"]]
print(X.shape)
print(y.shape)

(303, 13)
(303, 1)


## Split dataset

In [47]:
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=123)

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(212, 13)
(91, 13)
(212, 1)
(91, 1)


In [48]:
print(y_train.value_counts())
print(y_test.value_counts())

target
1         115
0          97
dtype: int64
target
1         50
0         41
dtype: int64


## Balance the Training Dataset

In [49]:
sm = SMOTE(random_state=123)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print(X_train_res.shape)
print(y_train_res.shape)

(230, 13)
(230, 1)


## Build Model

In [64]:
modelXGBM = xgb.XGBClassifier(colsample_bytree= 1, learning_rate=0.1, max_depth=4, 
                              min_child_weight=1e-05, n_estimators=200, 
                              objective='binary:logistic', subsample=0.5)

In [65]:
modelXGBM.fit(X_train,y_train)
model_prediction = modelXGBM.predict(X_test)

[21:38:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/paulo/Documents/workspace/heart-disease-classifier/venv/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/paulo/Documents/workspace/heart-disease-classifier/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

In [66]:

print(accuracy_score(y_test, model_prediction))
print(roc_auc_score(y_test, model_prediction))

0.7362637362637363
0.7314634146341463


## Persist Model and data used

In [68]:
joblib.dump(modelXGBM, "../data/modelXGBM.pkl")

['../data/modelXGBM.pkl']

In [70]:
X_train_res.to_csv("../data/X_train.csv")
X_test.to_csv("../data/X_test.csv")
y_train_res.to_csv("../data/y_train.csv")
y_test.to_csv("../data/y_test.csv")